In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
%matplotlib inline

## Loading the dataset

In [ ]:
zf_train = zipfile.ZipFile('/kaggle/input/whats-cooking/train.json.zip')
df_train = pd.read_json(zf_train.open('train.json'))

## Overview of the dataset

In [ ]:
print(df_train.head())
df_train = df_train.drop('id', axis = 1)

In [ ]:
print(df_train.head())
print(df_train.shape)

## What are the cuisines in the dataset

In [ ]:
cuisines = df_train.cuisine.unique()
print(cuisines.shape)

## Plotting the cuisine versus number of examples of that cuisine

In [ ]:
fig = plt.figure()
fig.set_size_inches(20, 6)
sns.barplot(data = df_train.groupby('cuisine').count(), x = df_train.groupby('cuisine').count().index, y = 'ingredients')

## Finding which ingredients are available in a cuisine

In [ ]:
cuisine_ingredients = {}
for i, cuisine in enumerate(cuisines):
    df = df_train[df_train['cuisine'] == cuisine]
    ingredients = []
    df['ingredients'].apply(lambda x : ingredients.extend(x))
    ingredients = np.array(ingredients)
    unique_ingredients = np.unique(list(ingredients))
    cuisine_ingredients[cuisine] = unique_ingredients
print(cuisine_ingredients)

## Total different types of ingredients used in all of the cuisines

In [ ]:
total_ingredients = []
for cuisine, ingredients in cuisine_ingredients.items():
    total_ingredients.extend(ingredients)
print(len(total_ingredients))
total_different_ingredients = list(np.unique(np.array(total_ingredients)))
print("Total different types of ingredients :", len(total_different_ingredients))

## Make every ingredient a column in the dataset

In [ ]:
def make_columns(dataframe):
    for ingredient in total_different_ingredients:
        dataframe[ingredient] = 0
    return dataframe

df_train = make_columns(df_train)
print(df_train.shape)

In [ ]:
df_train.head()

## Filling the columns according to the ingredients of the reciepies

In [ ]:
def fill_columns(dataframe):
    for index, row in dataframe.iterrows():
        ingredients = row['ingredients']
        print(index)
        for ingredient in ingredients:
            if ingredient in total_different_ingredients:
                dataframe.at[index, ingredient] = 1
    return dataframe

df_train = fill_columns(df_train)

## Finding the ingredients who are present in at least 2 percent receipies of a particular cuisine

In [ ]:
df_grouped = df_train.groupby('cuisine').mean()
df_grouped

In [ ]:
keep_percent = .2
top_cuisine_ingredients = {}
for cuisine in cuisines:
    a = df_grouped.loc[cuisine, :].sort_values() < keep_percent
    a = a[a == True]
    top_cuisine_ingredients[cuisine] = a.index

top_ingredients = []
for cuisine, ingredients in top_cuisine_ingredients.items():
    top_ingredients.extend(ingredients)
    
top_ingredients = np.array(top_ingredients)
unique_top_ingredients = ['cuisine']
unique_top_ingredients.extend(list(np.unique(top_ingredients)))
print(len(unique_top_ingredients))

In [ ]:
df_train = df_train[unique_top_ingredients]
print(df_train.shape)

## Top 5 ingredient in each cuisine

In [ ]:
temp = df_train.groupby('cuisine').mean()

In [ ]:
fig, ax = plt.subplots(10, 2)
ax = ax.flatten()
fig.set_size_inches(20, 50)
for index, cuisine in enumerate(cuisines):
    ax[index].title.set_text(cuisine)
    sns.barplot(x = top_cuisine_ingredients[cuisine][:-1][-5:], y = temp.loc[cuisine][top_cuisine_ingredients[cuisine]].values[:-1][-5:], ax = ax[index])

## Importing the Machine Learning Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Y = df_train.cuisine
X = df_train.drop(['cuisine'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.15)

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = height).fit(X_train, y_train)
test_predictions = rf.predict(X_test)
train_predictions = rf.predict(X_train)
print("height is ", height)
print("Training Accuracy is : ", sum(train_predictions==y_train)/y_train.shape[0])
print("Testing Accuracy is : ", sum(test_predictions==y_test)/y_test.shape[0])

## Preprocessing the test data

In [ ]:
zf_test = zipfile.ZipFile('/kaggle/input/whats-cooking/test.json.zip')
df_test = pd.read_json(zf_test.open('test.json'))
df_test.head()

In [ ]:
df_test = make_columns(df_test)
df_test.head()

In [ ]:
df_test = fill_columns(df_test)
df_test.head()

In [ ]:
id = df_test['id']
df_test = df_test[unique_top_ingredients[1:]]

In [ ]:
df_test.head()

In [ ]:
predictions = rf.predict(df_test)

In [ ]:
df = pd.DataFrame({'id':id, 'cuisine':predictions})

In [ ]:
df.to_csv('cooking_submission.csv', index = False)

# ** Direct method with python libraries instead of self made methods like make_columns and fill_columns**

In [ ]:
zf_train = zipfile.ZipFile('/kaggle/input/whats-cooking/train.json.zip')
df_train = pd.read_json(zf_train.open('train.json'))

In [ ]:
df_train.head()

## Converting the list of ingredient to a string

In [ ]:
def joiningre(x):
    return ' '.join(x)
df_train['all_ingredients'] = df_train['ingredients'].apply(joiningre)

In [ ]:
df_train.head(2)

## Using count vectorizer to build the vocab and then converting the ingredients string to a sparse matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(df_train['all_ingredients'])

In [ ]:
len(count_vect.vocabulary_)

In [ ]:
df_train_transform = count_vect.transform(df_train['all_ingredients'])

## Converting the sparse matrix to one hot encode matrix

In [ ]:
X = df_train_transform.toarray()
Y = df_train['cuisine']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.15)

In [ ]:
rf = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
test_predictions = rf.predict(X_test)
train_predictions = rf.predict(X_train)
print("height is ", height)
print("Training Accuracy is : ", sum(train_predictions==y_train)/y_train.shape[0])
print("Testing Accuracy is : ", sum(test_predictions==y_test)/y_test.shape[0])

In [ ]:
print(train_predictions)

## Predictions with the test data

In [ ]:
zf_test = zipfile.ZipFile('/kaggle/input/whats-cooking/test.json.zip')
df_test = pd.read_json(zf_test.open('test.json'))
df_test['all_ingredients'] = df_test['ingredients'].apply(joiningre)

In [ ]:
df_test.head()

In [ ]:
df_test_transform = count_vect.transform(df_test['all_ingredients'])

In [ ]:
predictions = rf.predict(df_test_transform.toarray())
print(predictions)

In [ ]:
df = pd.DataFrame({'id':df_test.id, 'cuisine':predictions})

In [ ]:
df.to_csv('cooking_submission.csv', index = False)